In [14]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas pyarrow wandb plotly


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
import math
import pandas as pd
import numpy as np
#Load parquet data into dataset_parquet for training.
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from glob import glob
from tqdm import tqdm
from collections import Counter
import random
import time
import wandb

In [32]:
class CONFIG:
    # root = os.path.join("/", "kaggle", "input", "asl-signs") 
    #root = os.path.join("..","data", "ASL-ds")
    root = os.path.join(".")
    DATA_LIMIT = 1000
    BATCH_SIZE = 16
    VIDEO_LENGTH = 15
    TRAIN_VAL_SPLIT = 0.85
    WANDB_RUN = "mediapipe-asl-dataset"


In [33]:
LIPS_IDXS0 = np.array([
        61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    ])

EYE_LEFT = np.array([33, 7, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 471, 470, 469, 472])
EYE_RIGHT = np.array([362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 476, 475, 474, 477])
# Landmark indices in original data
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)
LEFT_POSE_IDXS0 = np.array([502, 504, 506, 508, 510])
RIGHT_POSE_IDXS0 = np.array([503, 505, 507, 509, 511])

print("LIPS : ",len(LIPS_IDXS0))
print("EYE_LEFT : ",len(EYE_LEFT))
print("EYE_RIGHT : ",len(EYE_RIGHT))
print("LEFT_HAND : ",len(LEFT_HAND_IDXS0))
print("RIGHT_HAND : ",len(RIGHT_HAND_IDXS0))
print("LEFT_POSE : ",len(LEFT_POSE_IDXS0))
print("RIGHT_POSE : ",len(RIGHT_POSE_IDXS0))

all_selection = np.concatenate([LIPS_IDXS0, EYE_LEFT, EYE_RIGHT, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, LEFT_POSE_IDXS0, RIGHT_POSE_IDXS0])
print(len(all_selection))

LIPS :  40
EYE_LEFT :  20
EYE_RIGHT :  20
LEFT_HAND :  21
RIGHT_HAND :  21
LEFT_POSE :  5
RIGHT_POSE :  5
132


In [34]:
#this code sorts out a parquet files and rearrange the order to pose,face, left-hand, right-hand
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objs as go
import plotly.io as pio

ids = None

order_global = {"pose" : 10000, "face" : 1000, "left_hand" : 100, "right_hand" : 10}

def visualize_keypoints(frames : np.ndarray, point_size : int):
    if len(frames.shape) == 1:
        frames = np.array([frames])
    
    for frame in frames:
        frame = frame.reshape(-1, 3)
        sizes = point_size * np.ones(frame.shape[0])

        fig = go.Figure(data=go.Scatter(x=frame[:,0], y=2.5 - frame[:,1], mode='markers',
                                        marker=dict(
                                            size=sizes
                                            )))

    # Customize the layout
    fig.update_layout(title='visualization of human keypoints',
                        xaxis_title='',
                        yaxis_title='',
                        width=1000,
                        height=1600)

    fig.update_xaxes(range=[-0.2, 1.4])  # Set x-axis range from 0 to 6
    fig.update_yaxes(range=[0, 2.5])  # Set y-axis range from 10 to 20

    # Show the plot
    fig.show()


def process_parquet(ds, idxes = None):
    ret = []
    frames_unique = sorted(np.unique(ds["frame"]))
    for i,frame in enumerate(frames_unique):
        frame_ds = ds[ds['frame'] == frame]
        
        order = []
        for el in frame_ds["row_id"]:
            _frame, part, keypoint = el.split("-")
            order.append(order_global[part] - int(keypoint))

        order = np.array(order)
        frame_ds.iloc[:, 1] = order
        frame_ds = frame_ds.sort_values(by="row_id", ascending=False)
    
        vals = np.array(frame_ds[["x", "y", "z"]])
        if idxes is not None:
            vals = vals[idxes]
    
        vals = vals.flatten()

        ret.append(vals)
        
    return np.array(ret)


def process_parquet2(ds, idxes = None):
    ret = []    
    frame_size = 543
    it = len(ds) // frame_size
    assert it == len(ds) / frame_size
    
    for i in range(it):
        vals = ds.iloc[ i * frame_size : (i + 1 ) * frame_size ]        
        
        if idxes is not None:          
            vals = ds.iloc[idxes]
                        
        ret.append(np.array(vals[["x","y", "z"]]).flatten())
        
    return np.array(np.array(ret))

In [35]:
from sklearn.model_selection import train_test_split

#custom class to load data from Parquet files for training ML models.
class ParquetDatasetCached(keras.utils.Sequence):
    def __init__(self, dataset_folder, csv_file : str, batch_size=CONFIG.BATCH_SIZE, 
                 data_limit :int= CONFIG.DATA_LIMIT, check_if_file_exists = True, 
                 preprocessing_func=None, frame_length :int = CONFIG.VIDEO_LENGTH,
                 split : str = "train", train_val_split : float = CONFIG.TRAIN_VAL_SPLIT,
                 sort_by_counts : bool = True, **kwargs
                ):
        super().__init__(**kwargs)
        #taking keras sequence for .fit(), .evaluate(), .predict() methods
        #load csv - it has the path to parquet file, and another to store label
        self.csv_path = csv_file
        self.root_folder = dataset_folder
        self.batch_size = batch_size
        #optional pre-processing function to the parquet files.
        self.preprocessing_func = preprocessing_func
        
        self.csv_data = pd.read_csv(self.csv_path)
        
        self.all_files = []
        self.not_exists = []
        self.frame_length = frame_length

        
        for path, label in tqdm(list(zip(self.csv_data["path"], self.csv_data["sign"]))):
            prop_path = os.path.join(self.root_folder, path)
            
            if check_if_file_exists:
                if os.path.exists(prop_path):
                    self.all_files.append((prop_path, label))
                else:
                    self.not_exists.append(prop_path)
            else:
                self.all_files.append((prop_path, label))
                
                    
        self.all_files = np.array(self.all_files)
        self.unique_labels = np.unique(self.all_files[:, 1])
        self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
    
        # sort the values by popularity
        if sort_by_counts:
            cnt = Counter(self.all_files[:, 1])
            vals = []
            
            for i,row in enumerate(self.all_files):
                vals.append((int(1e6 * cnt[row[1]] + self.label_2_id [row[1]]),i))
            
            vals = np.array(sorted(vals)[::-1])
            self.all_files = self.all_files[vals[:,1]]

        
        if data_limit < 0:
            train_ds, val_ds = train_test_split(self.all_files, train_size=train_val_split, random_state=42)
        else:
            train_ds, val_ds = train_test_split(self.all_files[:data_limit], train_size=train_val_split, random_state=42)
            self.unique_labels = np.unique(self.all_files[:data_limit, 1])
            self.label_2_id = { key : i for i, key in enumerate(self.unique_labels)}
            
        if split.lower() == "train":
            self.dataset = train_ds
            
        elif split.lower() == "val":
            self.dataset = val_ds 
            
        else:
            raise Exception("please specify split to be either train or val")
            
        self.cache_data()
                   

    def cache_data(self):
        self.cached_X, self.cached_Y = [], []
        
        pb = tqdm(range(len(self.dataset)), desc="Cacheing")

        for i in pb:
            
            path, label = self.dataset[i]
            df = pd.read_parquet(path)
                        
            one_hot_encoded_label = np.zeros(len(self.unique_labels))
            one_hot_encoded_label[self.label_2_id[label]] = 1

            self.cached_X.append(process_parquet2(df)) 
            self.cached_Y.append(one_hot_encoded_label)        

    def __len__(self):
        # Assuming each Parquet file should be one batch; adjust if necessary
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.preprocessing_func(self.cached_X[idx]), self.cached_Y[idx]                
        
    def on_epoch_end(self):
        # Shuffle files for the next epoch
        np.random.shuffle(self.dataset)


def preprocess(frames):
    current_length, num_features = frames.shape

    if current_length >= CONFIG.VIDEO_LENGTH:
            # TODO: a better than uniform value ? Could place gaussian in the middle
            random_start = random.randint(0, current_length - CONFIG.VIDEO_LENGTH)
            return np.nan_to_num(frames[random_start : (random_start + CONFIG.VIDEO_LENGTH)])
        
    # padd the video to contain zeros 
    return np.concatenate([np.nan_to_num(frames), np.zeros((CONFIG.VIDEO_LENGTH - current_length, num_features))], axis=0)

    
# Usage example
parquet_folder_path = CONFIG.root
train_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                check_if_file_exists = True,
                                split="train")

val_dataset_parquet = ParquetDatasetCached(parquet_folder_path, csv_file = os.path.join(CONFIG.root, "train.csv"), 
                                 batch_size=CONFIG.BATCH_SIZE, data_limit=CONFIG.DATA_LIMIT,
                                 preprocessing_func=preprocess,
                                 check_if_file_exists= True,
                                 split="val")

print(f"cardinality of train : {len(train_dataset_parquet)}, cardinality of validation : {len(val_dataset_parquet)}")

Cacheing: 100%|██████████| 150/150 [00:02<00:00, 54.47it/s]

cardinality of train : 850, cardinality of validation : 150


In [36]:
X_shape = train_dataset_parquet[0][0].shape
Y_shape = train_dataset_parquet[0][1].shape
print(f"X_shape = {X_shape}, Y_shape = {Y_shape}")

X_shape = (15, 1629), Y_shape = (3,)


In [37]:
print(train_dataset_parquet.label_2_id)

{'listen': 0, 'look': 1, 'shhh': 2}


In [7]:
def dataset_train_generator():
    # Instantiate your existing dataset loader

    for i in range(len(train_dataset_parquet)):
        X_batch, Y_batch = train_dataset_parquet[i]
        yield X_batch, Y_batch
        
def dataset_val_generator():
    # Instantiate your existing dataset loader

    for i in range(len(val_dataset_parquet)):
        X_batch, Y_batch = val_dataset_parquet[i]
        yield X_batch, Y_batch        


train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_train_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_val_generator(),
    output_types=(tf.float32, tf.float32),  # Adjust types based on your actual data
    output_shapes=(X_shape, Y_shape)
).prefetch(tf.data.AUTOTUNE).batch(CONFIG.BATCH_SIZE)

2024-04-22 11:56:42.007624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22280 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:04:00.0, compute capability: 8.6


In [8]:
start = time.time()
isnans =False

f = True
labels_batches = []
for el in tqdm(train_dataset):
    if f:
        print(el[0].shape, el[1].shape)
        f = False
    labels_batches.append(el[1])
        
    isnans |= np.any(np.isnan(el[0]))
    if isnans:
        print("FOUND NAN!")
        break


print(f"Iterating through dataset took : {round( time.time() - start , 4)}s")

25it [00:00, 123.77it/s]

(16, 15, 1629) (16, 3)


54it [00:00, 122.28it/s]

Iterating through dataset took : 0.4454s


# Train experiment

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from wandb.keras import WandbMetricsLogger
from keras.callbacks import Callback
import tensorflow as tf
import wandb


class CosineAnnealingLearningRateScheduler(Callback):
    def __init__(self, max_lr, min_lr, T_max):
        super(CosineAnnealingLearningRateScheduler, self).__init__()
        self.max_lr = max_lr  # Maximum learning rate (i.e., start learning rate)
        self.min_lr = min_lr  # Minimum learning rate
        self.T_max = T_max    # Specifies the number of epochs per cycle
        self.t = 0            # Current epoch

    def on_epoch_begin(self, epoch, logs=None):
        self.t += 1
        cos = np.cos(np.pi * (self.t % self.T_max) / self.T_max)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + cos)

        keras.backend.set_value(self.model.optimizer.lr, lr)

def keras_train(model, filepath : str, max_lr = 1e-4, min_lr = 2.5e-5, T_max=101, epochs=200, run_name="",
                mediapipe_features = "all", USE_WANDB=True,
               verbose="auto"): 
    
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                 monitor="val_categorical_accuracy",
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode="max",
                                                 save_freq="epoch")
    
    cosine_annealer = CosineAnnealingLearningRateScheduler(max_lr=max_lr,
                                                           min_lr=min_lr,
                                                           T_max=T_max)
    
    #Adam Optimizer - fixed learning rate.
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=max_lr, clipnorm=1.)

    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    
    callbacks  = [checkpoint, cosine_annealer]
    
    if USE_WANDB:
        wandb.init(project=CONFIG.WANDB_RUN,
                        name=run_name,
                        notes="Model summary : \n" + str(model),
                        config={"max_lr" : max_lr, 
                                "min_lr" : 5e-5, 
                                "scheduler" : "cosineAnnealer", 
                                "epochs" : epochs, 
                                "T_max" : T_max, 
                                "train_size" : len(train_dataset_parquet.dataset),
                                "val_size" : len(val_dataset_parquet.dataset),
                                "unique_classes" : len(train_dataset_parquet.unique_labels), 
                                "video_length" : CONFIG.VIDEO_LENGTH,
                                "features" : mediapipe_features
                                })
        callbacks.append(WandbMetricsLogger())


    
    history = model.fit(train_dataset, epochs=epochs, validation_data = val_dataset, callbacks=callbacks, verbose=verbose)
    
    if USE_WANDB:      
        wandb.finish()
    
    return history

## LSTM

In [28]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM16-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM16-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)

wandb: Currently logged in as: mlewand (mlewand7). Use `wandb login --relogin` to force relogin


Epoch 1/100
     47/Unknown - 3s 7ms/step - loss: 1.0794 - categorical_accuracy: 0.4109

2024-04-22 11:51:00.155827: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12364882914382341265
2024-04-22 11:51:00.155958: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4272306358147487261
2024-04-22 11:51:00.156004: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4734228563005901122
2024-04-22 11:51:00.841412: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16827236687576336969
2024-04-22 11:51:00.841454: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3824467376321958737
2024-04-22 11:51:00.841462: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12161850947753172941
2024-04-22 11:51:00.841478: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 128ms/step - loss: 1.0588 - categorical_accuracy: 0.4259 - val_loss: 1.0595 - val_categorical_accuracy: 0.4533
Epoch 3/100
54/54 [==============================] - 0s 9ms/step - loss: 1.0532 - categorical_accuracy: 0.4576 - val_loss: 1.0546 - val_categorical_accuracy: 0.4333
Epoch 4/100
54/54 [==============================] - 0s 9ms/step - loss: 1.0466 - categorical_accuracy: 0.4624 - val_loss: 1.0479 - val_categorical_accuracy: 0.4467
Epoch 5/100
52/54 [===========================>..] - ETA: 0s - loss: 1.0382 - categorical_accuracy: 0.4651INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM16-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 129ms/step - loss: 1.0369 - categorical_accuracy: 0.4659 - val_loss: 1.0429 - val_categorical_accuracy: 0.5267
Epoch 6/100
54/54 [==============================] - 1s 10ms/step - loss: 1.0302 - categorical_accuracy: 0.4647 - val_loss: 1.0157 - val_categorical_accuracy: 0.5067
Epoch 7/100
54/54 [==============================] - 1s 9ms/step - loss: 1.0009 - categorical_accuracy: 0.4988 - val_loss: 1.0249 - val_categorical_accuracy: 0.5067
Epoch 8/100
54/54 [==============================] - 1s 10ms/step - loss: 0.9896 - categorical_accuracy: 0.5259 - val_loss: 0.9525 - val_categorical_accuracy: 0.5133
Epoch 9/100
47/54 [=========================>....] - ETA: 0s - loss: 0.9427 - categorical_accuracy: 0.5678


KeyboardInterrupt



In [10]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM32-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM32-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)

2024-04-22 11:56:42.796543: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
wandb: Currently logged in as: mlewand (mlewand7). Use `wandb login --relogin` to force relogin


Epoch 1/200


2024-04-22 11:56:48.939619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-04-22 11:56:50.344025: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f262410ffb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 11:56:50.344056: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-22 11:56:50.348231: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-22 11:56:50.440136: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


     52/Unknown - 6s 7ms/step - loss: 1.0907 - categorical_accuracy: 0.4315

2024-04-22 11:56:51.218564: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15961899707335931791
2024-04-22 11:56:51.218616: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 11:56:51.218697: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762


...

INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 131ms/step - loss: 0.2384 - categorical_accuracy: 0.9224 - val_loss: 0.3143 - val_categorical_accuracy: 0.9133
Epoch 166/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2512 - categorical_accuracy: 0.9106 - val_loss: 0.3480 - val_categorical_accuracy: 0.8933
Epoch 167/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2205 - categorical_accuracy: 0.9165 - val_loss: 0.3881 - val_categorical_accuracy: 0.8733
Epoch 168/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2355 - categorical_accuracy: 0.9176 - val_loss: 0.4133 - val_categorical_accuracy: 0.8667
Epoch 169/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2474 - categorical_accuracy: 0.9188 - val_loss: 0.4380 - val_categorical_accuracy: 0.8667
Epoch 170/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2451 - categorical_accuracy: 0.9141 - val_loss: 0.3360 - val_categorical_accuracy: 0.8800
Epoch 1

INFO:tensorflow:Assets written to: models/AK-LSTM32-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 131ms/step - loss: 0.2199 - categorical_accuracy: 0.9259 - val_loss: 0.2610 - val_categorical_accuracy: 0.9267
Epoch 184/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1995 - categorical_accuracy: 0.9271 - val_loss: 0.2966 - val_categorical_accuracy: 0.9000
Epoch 185/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2311 - categorical_accuracy: 0.9247 - val_loss: 0.3611 - val_categorical_accuracy: 0.9067
Epoch 186/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2265 - categorical_accuracy: 0.9247 - val_loss: 0.3578 - val_categorical_accuracy: 0.8933
Epoch 187/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2306 - categorical_accuracy: 0.9294 - val_loss: 0.3264 - val_categorical_accuracy: 0.8800
Epoch 188/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2295 - categorical_accuracy: 0.9318 - val_loss: 0.2845 - val_categorical_accuracy: 0.9133
Epoch 189

epoch/categorical_accuracy,▁▁▃▄▄▅▆▆▆▇▇▇▇▇██████▇█▇█████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,██▇▆▅▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▂▂▂▄▄▅▆█▆▇█▆▇█▇█▇▇▇▆▇▆▆█▇▇▇▇▇▇▇▇▇██████
epoch/val_loss,██▆▆▅▅▄▃▂▃▃▂▃▂▂▂▂▃▂▂▃▂▄▃▁▂▃▂▂▂▂▂▂▂▁▁▁▁▂▁
epoch/categorical_accuracy,0.92941
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.20024
epoch/val_categorical_accuracy,0.9


In [11]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM64-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM64-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     54/Unknown - 3s 7ms/step - loss: 1.0953 - categorical_accuracy: 0.4318

2024-04-22 12:00:57.885436: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:00:57.885505: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 12:00:58.544845: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:00:58.544893: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 10s 143ms/step - loss: 1.0953 - categorical_accuracy: 0.4318 - val_loss: 1.0838 - val_categorical_accuracy: 0.4333
Epoch 2/200
50/54 [==========================>...] - ETA: 0s - loss: 1.0784 - categorical_accuracy: 0.4462INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 132ms/step - loss: 0.2668 - categorical_accuracy: 0.9082 - val_loss: 0.3121 - val_categorical_accuracy: 0.9133
Epoch 93/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2951 - categorical_accuracy: 0.9000 - val_loss: 0.3385 - val_categorical_accuracy: 0.8800
Epoch 94/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2824 - categorical_accuracy: 0.9047 - val_loss: 0.3782 - val_categorical_accuracy: 0.8867
Epoch 95/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2682 - categorical_accuracy: 0.9165 - val_loss: 0.3968 - val_categorical_accuracy: 0.8533
Epoch 96/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2711 - categorical_accuracy: 0.9200 - val_loss: 0.3689 - val_categorical_accuracy: 0.8867
Epoch 97/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2740 - categorical_accuracy: 0.9129 - val_loss: 0.3758 - val_categorical_accuracy: 0.8867
Epoch 98/200

INFO:tensorflow:Assets written to: models/AK-LSTM64-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 134ms/step - loss: 0.2412 - categorical_accuracy: 0.9365 - val_loss: 0.2423 - val_categorical_accuracy: 0.9200
Epoch 186/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2083 - categorical_accuracy: 0.9353 - val_loss: 0.3621 - val_categorical_accuracy: 0.8733
Epoch 187/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2127 - categorical_accuracy: 0.9412 - val_loss: 0.3258 - val_categorical_accuracy: 0.8733
Epoch 188/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2240 - categorical_accuracy: 0.9376 - val_loss: 0.3392 - val_categorical_accuracy: 0.8733
Epoch 189/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1943 - categorical_accuracy: 0.9482 - val_loss: 0.3760 - val_categorical_accuracy: 0.8800
Epoch 190/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1986 - categorical_accuracy: 0.9412 - val_loss: 0.3511 - val_categorical_accuracy: 0.8733
Epoch 1

epoch/categorical_accuracy,▁▂▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇███▇▇▇▇██▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▇▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▆▆▅▅▆▇▆▆▆▇▆▇▇▇▇▇█▇▆▅▇▇█▅▇█▇▆▆█▇▇▇▇█▇██
epoch/val_loss,█▇▄▄▄▅▃▃▃▄▃▂▃▃▃▂▂▂▁▂▃▅▂▃▁▇▂▁▃▃▄▂▂▂▂▂▂▁▁▂
epoch/categorical_accuracy,0.93294
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.22971
epoch/val_categorical_accuracy,0.89333


In [12]:
model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM128-LR-Dense256-fs=15-cls=3.tf"),
            run_name="AK-LSTM128-LR-Dense256-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     52/Unknown - 3s 7ms/step - loss: 1.1265 - categorical_accuracy: 0.4387

2024-04-22 12:04:47.973530: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:04:48.637253: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 10s 144ms/step - loss: 1.1258 - categorical_accuracy: 0.4353 - val_loss: 1.1049 - val_categorical_accuracy: 0.4467
Epoch 2/200
54/54 [==============================] - ETA: 0s - loss: 1.0908 - categorical_accuracy: 0.4612INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 125ms/step - loss: 0.3530 - categorical_accuracy: 0.8800 - val_loss: 0.3265 - val_categorical_accuracy: 0.9133
Epoch 114/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3198 - categorical_accuracy: 0.9035 - val_loss: 0.4602 - val_categorical_accuracy: 0.8800
Epoch 115/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2787 - categorical_accuracy: 0.9165 - val_loss: 0.4786 - val_categorical_accuracy: 0.8467
Epoch 116/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3533 - categorical_accuracy: 0.8859 - val_loss: 0.9248 - val_categorical_accuracy: 0.7067
Epoch 117/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3043 - categorical_accuracy: 0.9165 - val_loss: 0.5704 - val_categorical_accuracy: 0.8267
Epoch 118/200
54/54 [==============================] - 0s 9ms/step - loss: 0.3045 - categorical_accuracy: 0.9106 - val_loss: 0.4980 - val_categorical_accuracy: 0.8067
Epoch 11

INFO:tensorflow:Assets written to: models/AK-LSTM128-LR-Dense256-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 133ms/step - loss: 0.1870 - categorical_accuracy: 0.9494 - val_loss: 0.3712 - val_categorical_accuracy: 0.9200
Epoch 187/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2401 - categorical_accuracy: 0.9306 - val_loss: 0.4322 - val_categorical_accuracy: 0.8600
Epoch 188/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2074 - categorical_accuracy: 0.9400 - val_loss: 0.3966 - val_categorical_accuracy: 0.8933
Epoch 189/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2136 - categorical_accuracy: 0.9459 - val_loss: 0.4627 - val_categorical_accuracy: 0.8667
Epoch 190/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2087 - categorical_accuracy: 0.9459 - val_loss: 0.4223 - val_categorical_accuracy: 0.8800
Epoch 191/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2126 - categorical_accuracy: 0.9412 - val_loss: 0.3900 - val_categorical_accuracy: 0.8933
Epoch 

epoch/categorical_accuracy,▁▄▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇▇▇▇▇▇▇▇█▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁
epoch/val_categorical_accuracy,▁▆▆▆▆▅▇▅▇█▇▇████████▅▇▇▇▇▇▆▇▇▇▇█████▇███
epoch/val_loss,█▅▃▄▃▆▃▆▃▁▃▃▂▁▂▂▂▁▁▂▇▂▂▂▃▂▃▂▂▂▂▁▂▂▂▁▂▁▂▁
epoch/categorical_accuracy,0.96
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.17319
epoch/val_categorical_accuracy,0.89333


In [13]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     52/Unknown - 3s 7ms/step - loss: 1.1490 - categorical_accuracy: 0.4712

2024-04-22 12:08:32.316673: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:08:32.316738: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 12:08:32.316754: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17587355101111921732
2024-04-22 12:08:32.316786: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762
2024-04-22 12:08:32.974775: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11766465862500264075
2024-04-22 12:08:32.974839: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2540840263756146154


INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 11s 147ms/step - loss: 1.1446 - categorical_accuracy: 0.4729 - val_loss: 1.0881 - val_categorical_accuracy: 0.5333
Epoch 2/200
51/54 [===========================>..] - ETA: 0s - loss: 0.9802 - categorical_accuracy: 0.5833INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 135ms/step - loss: 0.2730 - categorical_accuracy: 0.9329 - val_loss: 0.4347 - val_categorical_accuracy: 0.9133
Epoch 114/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2935 - categorical_accuracy: 0.9212 - val_loss: 0.4448 - val_categorical_accuracy: 0.8733
Epoch 115/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2990 - categorical_accuracy: 0.9259 - val_loss: 1.0968 - val_categorical_accuracy: 0.6867
Epoch 116/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3253 - categorical_accuracy: 0.9106 - val_loss: 0.8225 - val_categorical_accuracy: 0.7467
Epoch 117/200
54/54 [==============================] - 0s 8ms/step - loss: 0.3556 - categorical_accuracy: 0.9012 - val_loss: 0.6504 - val_categorical_accuracy: 0.7733
Epoch 118/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2595 - categorical_accuracy: 0.9353 - val_loss: 0.4707 - val_categorical_accuracy: 0.8800
Epoch 119

INFO:tensorflow:Assets written to: models/AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 135ms/step - loss: 0.1617 - categorical_accuracy: 0.9718 - val_loss: 0.2830 - val_categorical_accuracy: 0.9333
Epoch 194/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1640 - categorical_accuracy: 0.9741 - val_loss: 0.3704 - val_categorical_accuracy: 0.9000
Epoch 195/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1533 - categorical_accuracy: 0.9753 - val_loss: 0.3704 - val_categorical_accuracy: 0.9133
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1518 - categorical_accuracy: 0.9682 - val_loss: 0.3862 - val_categorical_accuracy: 0.8867
Epoch 197/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1631 - categorical_accuracy: 0.9694 - val_loss: 0.3253 - val_categorical_accuracy: 0.9000
Epoch 198/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1584 - categorical_accuracy: 0.9647 - val_loss: 0.3636 - val_categorical_accuracy: 0.9133
Epoch 

epoch/categorical_accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇████▇▇▇▇▇▇▇█▇▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▃▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▂▅▆▆▆▁▆▇▇▆▄▆▇▆▆▆▆███▆▆▇▇▆▆▄▇▅▆▆█▇▇███▇██
epoch/val_loss,▄▃▃▂▂█▂▂▃▃▅▂▂▂▃▃▂▁▁▂▂▃▂▂▄▃▄▂▄▂▃▂▂▃▁▁▁▂▁▁
epoch/categorical_accuracy,0.96471
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16504
epoch/val_categorical_accuracy,0.88667


In [14]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(LSTM(512, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf"),
            run_name="AK-LSTM256-LR-Dense128-gelu-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     54/Unknown - 3s 7ms/step - loss: 1.2112 - categorical_accuracy: 0.4541

2024-04-22 12:11:58.431599: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17221516245317488493
2024-04-22 12:11:58.431670: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762
2024-04-22 12:11:59.098825: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:11:59.098877: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2540840263756146154
2024-04-22 12:11:59.098892: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2307519905931378332
2024-04-22 12:11:59.098922: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 11s 148ms/step - loss: 1.2112 - categorical_accuracy: 0.4541 - val_loss: 1.0822 - val_categorical_accuracy: 0.5533
Epoch 2/200
51/54 [===========================>..] - ETA: 0s - loss: 0.9898 - categorical_accuracy: 0.5980INFO:tensorflow:Assets written to: models/AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 128ms/step - loss: 0.2470 - categorical_accuracy: 0.9494 - val_loss: 0.5232 - val_categorical_accuracy: 0.9000
Epoch 88/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2250 - categorical_accuracy: 0.9600 - val_loss: 0.4774 - val_categorical_accuracy: 0.8733
Epoch 89/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2378 - categorical_accuracy: 0.9518 - val_loss: 0.5030 - val_categorical_accuracy: 0.8867
Epoch 90/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2620 - categorical_accuracy: 0.9412 - val_loss: 0.5158 - val_categorical_accuracy: 0.8467
Epoch 91/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2240 - categorical_accuracy: 0.9600 - val_loss: 0.6174 - val_categorical_accuracy: 0.8533
Epoch 92/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2683 - categorical_accuracy: 0.9447 - val_loss: 0.5105 - val_categorical_accuracy: 0.8933
Epoch 93/20

INFO:tensorflow:Assets written to: models/AK-LSTM512-LR-Dense128-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 137ms/step - loss: 0.1744 - categorical_accuracy: 0.9659 - val_loss: 0.3910 - val_categorical_accuracy: 0.9067
Epoch 175/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1661 - categorical_accuracy: 0.9647 - val_loss: 0.5136 - val_categorical_accuracy: 0.8667
Epoch 176/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1642 - categorical_accuracy: 0.9682 - val_loss: 0.5032 - val_categorical_accuracy: 0.8667
Epoch 177/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1659 - categorical_accuracy: 0.9694 - val_loss: 0.4618 - val_categorical_accuracy: 0.8533
Epoch 178/200
54/54 [==============================] - 0s 9ms/step - loss: 0.1681 - categorical_accuracy: 0.9647 - val_loss: 0.4739 - val_categorical_accuracy: 0.8667
Epoch 179/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1599 - categorical_accuracy: 0.9694 - val_loss: 0.4598 - val_categorical_accuracy: 0.8667
Epoch 18

epoch/categorical_accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▂▅▆▇▄▆▆▆▆▃▅█▆▇▇▇██▆▇▆▇▆▅▁▃▆▆▇▆▇▅█▇█▇▇▇
epoch/val_loss,▆▄▅▄▃▃▆▃▄▄▃▇▅▂▃▂▃▂▃▂▂▂▃▃▃▃█▆▃▃▂▃▂▄▂▂▁▂▂▂
epoch/categorical_accuracy,0.97412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.15444
epoch/val_categorical_accuracy,0.87333


## GRU

In [15]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(16, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf"),
            run_name="AK-GRU16-LR-Dense256-gelu-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 4s 7ms/step - loss: 1.0859 - categorical_accuracy: 0.4130

2024-04-22 12:15:39.118358: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:15:39.118424: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9560581064182111562
2024-04-22 12:15:39.118440: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2540840263756146154
2024-04-22 12:15:39.118475: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 10s 125ms/step - loss: 1.0827 - categorical_accuracy: 0.4059 - val_loss: 1.0683 - val_categorical_accuracy: 0.4133
Epoch 2/200
46/54 [========================>.....] - ETA: 0s - loss: 1.0677 - categorical_accuracy: 0.3940INFO:tensorflow:Assets written to: models/AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 6s 118ms/step - loss: 0.2539 - categorical_accuracy: 0.9188 - val_loss: 0.2863 - val_categorical_accuracy: 0.9133
Epoch 164/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2478 - categorical_accuracy: 0.9176 - val_loss: 0.4250 - val_categorical_accuracy: 0.8600
Epoch 165/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2728 - categorical_accuracy: 0.9082 - val_loss: 0.3464 - val_categorical_accuracy: 0.9000
Epoch 166/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2627 - categorical_accuracy: 0.9153 - val_loss: 0.3765 - val_categorical_accuracy: 0.8667
Epoch 167/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2581 - categorical_accuracy: 0.9094 - val_loss: 0.2438 - val_categorical_accuracy: 0.9133
Epoch 168/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2498 - categorical_accuracy: 0.9106 - val_loss: 0.3560 - val_categorical_accuracy: 0.8800
Epoch 

INFO:tensorflow:Assets written to: models/AK-GRU16-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 130ms/step - loss: 0.2984 - categorical_accuracy: 0.8941 - val_loss: 0.2338 - val_categorical_accuracy: 0.9333
Epoch 182/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2598 - categorical_accuracy: 0.9106 - val_loss: 0.3544 - val_categorical_accuracy: 0.8800
Epoch 183/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2734 - categorical_accuracy: 0.9094 - val_loss: 0.2339 - val_categorical_accuracy: 0.9333
Epoch 184/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2543 - categorical_accuracy: 0.9071 - val_loss: 0.2191 - val_categorical_accuracy: 0.9333
Epoch 185/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2274 - categorical_accuracy: 0.9212 - val_loss: 0.3101 - val_categorical_accuracy: 0.8933
Epoch 186/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2633 - categorical_accuracy: 0.9082 - val_loss: 0.2988 - val_categorical_accuracy: 0.8933
Epoch 18

epoch/categorical_accuracy,▁▂▄▄▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,██▇▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▂▂▃▄▃▅▅▆▄▆▆▆▆▆▇▇▇▇▇▆▆▇▅▆▇▆▆▆▇▇▇▇▇██████
epoch/val_loss,██▇▇▆▆▅▅▄▅▃▄▃▄▃▃▂▂▃▃▃▄▃▄▄▃▃▄▄▂▂▂▃▂▂▂▁▂▁▁
epoch/categorical_accuracy,0.91412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.25144
epoch/val_categorical_accuracy,0.92667


In [16]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(32, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf"),
            run_name="AK-GRU32-LR-Dense256-gelu-fs=15-cls=3",
            USE_WANDB=True)

Epoch 1/200
     46/Unknown - 3s 7ms/step - loss: 1.0838 - categorical_accuracy: 0.4511

2024-04-22 12:19:55.908596: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1956469891486031824
2024-04-22 12:19:55.908642: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:19:55.908661: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762
2024-04-22 12:19:56.511865: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:19:56.511930: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 9s 128ms/step - loss: 1.0772 - categorical_accuracy: 0.4565 - val_loss: 1.0731 - val_categorical_accuracy: 0.4133
Epoch 2/200
51/54 [===========================>..] - ETA: 0s - loss: 1.0687 - categorical_accuracy: 0.4534INFO:tensorflow:Assets written to: models/AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 7s 130ms/step - loss: 0.2449 - categorical_accuracy: 0.9235 - val_loss: 0.2283 - val_categorical_accuracy: 0.9333
Epoch 173/200
54/54 [==============================] - 0s 8ms/step - loss: 0.2075 - categorical_accuracy: 0.9318 - val_loss: 0.3276 - val_categorical_accuracy: 0.8800
Epoch 174/200
54/54 [==============================] - 0s 9ms/step - loss: 0.1926 - categorical_accuracy: 0.9388 - val_loss: 0.3158 - val_categorical_accuracy: 0.8800
Epoch 175/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2350 - categorical_accuracy: 0.9212 - val_loss: 0.2874 - val_categorical_accuracy: 0.8867
Epoch 176/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2320 - categorical_accuracy: 0.9247 - val_loss: 0.3305 - val_categorical_accuracy: 0.8933
Epoch 177/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1940 - categorical_accuracy: 0.9365 - val_loss: 0.3328 - val_categorical_accuracy: 0.9000
Epoch 178/

INFO:tensorflow:Assets written to: models/AK-GRU32-LR-Dense256-gelu-fs=15-cls=3.tf/assets


54/54 [==============================] - 6s 118ms/step - loss: 0.2150 - categorical_accuracy: 0.9259 - val_loss: 0.2313 - val_categorical_accuracy: 0.9467
Epoch 197/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2172 - categorical_accuracy: 0.9294 - val_loss: 0.2714 - val_categorical_accuracy: 0.9067
Epoch 198/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1904 - categorical_accuracy: 0.9365 - val_loss: 0.2653 - val_categorical_accuracy: 0.9000
Epoch 199/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1871 - categorical_accuracy: 0.9376 - val_loss: 0.2030 - val_categorical_accuracy: 0.9400
Epoch 200/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1979 - categorical_accuracy: 0.9376 - val_loss: 0.3370 - val_categorical_accuracy: 0.8867


epoch/categorical_accuracy,▁▂▄▄▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇████▇███████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,██▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▂▃▄▆▆▇▆▇▇▆▇█▇▇▇████▇█▇█▇▇▇█▇▇▇███▇▇█▇█▇
epoch/val_loss,██▆▆▅▄▃▄▃▂▃▃▂▂▂▁▂▂▂▂▂▂▁▂▂▃▂▁▂▂▂▁▁▁▂▂▂▂▁▂
epoch/categorical_accuracy,0.93765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.19787
epoch/val_categorical_accuracy,0.88667


In [17]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(64, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU64-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-GRU64-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     52/Unknown - 3s 7ms/step - loss: 1.0917 - categorical_accuracy: 0.4423

2024-04-22 12:24:02.008528: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 12:24:02.627033: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:24:02.627073: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17381370266973075638
2024-04-22 12:24:02.627081: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2540840263756146154
2024-04-22 12:24:02.627104: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-GRU64-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 9s 129ms/step - loss: 1.0910 - categorical_accuracy: 0.4388 - val_loss: 1.0844 - val_categorical_accuracy: 0.4133
Epoch 2/200
49/54 [==========================>...] - ETA: 0s - loss: 1.0779 - categorical_accuracy: 0.4388INFO:tensorflow:Assets written to: models/AK-GRU64-LR-Dense256-gelu-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 6s 120ms/step - loss: 0.3354 - categorical_accuracy: 0.8812 - val_loss: 0.2704 - val_categorical_accuracy: 0.9267
Epoch 109/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3265 - categorical_accuracy: 0.8847 - val_loss: 0.3004 - val_categorical_accuracy: 0.9067
Epoch 110/200
54/54 [==============================] - 0s 9ms/step - loss: 0.3003 - categorical_accuracy: 0.9024 - val_loss: 0.3398 - val_categorical_accuracy: 0.8800
Epoch 111/200
54/54 [==============================] - 0s 9ms/step - loss: 0.3289 - categorical_accuracy: 0.8882 - val_loss: 0.3027 - val_categorical_accuracy: 0.8867
Epoch 112/200
54/54 [==============================] - 0s 9ms/step - loss: 0.3442 - categorical_accuracy: 0.8894 - val_loss: 0.2820 - val_categorical_accuracy: 0.9200
Epoch 113/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2567 - categorical_accuracy: 0.9212 - val_loss: 0.3391 - val_categorical_accuracy: 0.8800
Epoch 114

INFO:tensorflow:Assets written to: models/AK-GRU64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 6s 119ms/step - loss: 0.2015 - categorical_accuracy: 0.9412 - val_loss: 0.2344 - val_categorical_accuracy: 0.9400
Epoch 195/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2277 - categorical_accuracy: 0.9329 - val_loss: 0.2782 - val_categorical_accuracy: 0.9200
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2151 - categorical_accuracy: 0.9329 - val_loss: 0.3209 - val_categorical_accuracy: 0.9200
Epoch 197/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2372 - categorical_accuracy: 0.9247 - val_loss: 0.2406 - val_categorical_accuracy: 0.9200
Epoch 198/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2314 - categorical_accuracy: 0.9224 - val_loss: 0.2913 - val_categorical_accuracy: 0.9200
Epoch 199/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2376 - categorical_accuracy: 0.9282 - val_loss: 0.2851 - val_categorical_accuracy: 0.9133
Epoch 200

epoch/categorical_accuracy,▁▃▄▅▆▆▇▇▇▇▆▇▇▇▇▇███▇▇▇▇▇▇▇█▇▇███▇███████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▇▅▄▄▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▆▇▆▇▇▇▇▆▇▇▆█▇▇▇▇▇▆██▇▇▇▇▇▇█▇▇▇▇█▇████
epoch/val_loss,█▇▆▄▃▃▃▃▂▂▄▃▂▄▁▂▂▂▂▂▄▁▁▂▂▂▂▂▂▁▃▂▂▂▂▂▁▂▁▂
epoch/categorical_accuracy,0.93059
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.22134
epoch/val_categorical_accuracy,0.91333


In [18]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(128, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU128-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-GRU128-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 3s 7ms/step - loss: 1.1227 - categorical_accuracy: 0.4632

2024-04-22 12:27:57.889681: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 12:27:57.889742: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:27:57.889759: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15667462077972524050
2024-04-22 12:27:57.889792: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762
2024-04-22 12:27:58.525716: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3783105877637645511
2024-04-22 12:27:58.525767: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930
2024-04-22 12:27:58.525780: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv

INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 10s 146ms/step - loss: 1.1203 - categorical_accuracy: 0.4576 - val_loss: 1.1149 - val_categorical_accuracy: 0.4400
Epoch 2/200
53/54 [============================>.] - ETA: 0s - loss: 1.0894 - categorical_accuracy: 0.4800INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 122ms/step - loss: 0.3353 - categorical_accuracy: 0.8906 - val_loss: 0.2457 - val_categorical_accuracy: 0.9267
Epoch 129/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3293 - categorical_accuracy: 0.9024 - val_loss: 0.4830 - val_categorical_accuracy: 0.8200
Epoch 130/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2685 - categorical_accuracy: 0.9165 - val_loss: 0.4020 - val_categorical_accuracy: 0.8600
Epoch 131/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3024 - categorical_accuracy: 0.9047 - val_loss: 0.4927 - val_categorical_accuracy: 0.8400
Epoch 132/200
49/54 [==========================>...] - ETA: 0s - loss: 0.2650 - categorical_accuracy: 0.9171INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU128-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 122ms/step - loss: 0.2673 - categorical_accuracy: 0.9165 - val_loss: 0.2495 - val_categorical_accuracy: 0.9333
Epoch 133/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2849 - categorical_accuracy: 0.9082 - val_loss: 0.4983 - val_categorical_accuracy: 0.8267
Epoch 134/200
54/54 [==============================] - 0s 9ms/step - loss: 0.2726 - categorical_accuracy: 0.9200 - val_loss: 0.4700 - val_categorical_accuracy: 0.8333
Epoch 135/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2449 - categorical_accuracy: 0.9259 - val_loss: 0.4496 - val_categorical_accuracy: 0.8467
Epoch 136/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2402 - categorical_accuracy: 0.9282 - val_loss: 0.3703 - val_categorical_accuracy: 0.8800
Epoch 137/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2729 - categorical_accuracy: 0.9224 - val_loss: 0.3934 - val_categorical_accuracy: 0.8667
Epoch 138

epoch/categorical_accuracy,▁▃▅▆▇▆▇▇▆▇▇▇█▇▇▇██▇█▇▇▇▇▇▇█▇▇▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▇▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁
epoch/val_categorical_accuracy,▁▁▅▄▆▇▆▇▄▃█▅▇▅▇▇▇▇▇▇▇▄▄▆▇█▆▄▇█▇▇█▇▇██▇▇▇
epoch/val_loss,█▇▅▄▃▃▃▂▄▆▂▄▂▄▂▃▂▃▂▂▂▆▆▃▂▁▃▇▂▁▂▂▁▂▂▂▂▂▂▂
epoch/categorical_accuracy,0.94706
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18963
epoch/val_categorical_accuracy,0.9


In [19]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(256, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU256-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-GRU256-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     48/Unknown - 3s 8ms/step - loss: 1.1637 - categorical_accuracy: 0.4453INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 10s 132ms/step - loss: 1.1552 - categorical_accuracy: 0.4471 - val_loss: 1.1551 - val_categorical_accuracy: 0.5267
Epoch 2/200
54/54 [==============================] - 1s 9ms/step - loss: 1.0990 - categorical_accuracy: 0.5188 - val_loss: 1.1399 - val_categorical_accuracy: 0.4600
Epoch 3/200
54/54 [==============================] - 1s 9ms/step - loss: 1.0457 - categorical_accuracy: 0.5365 - val_loss: 1.1261 - val_categorical_accuracy: 0.4733
Epoch 4/200
54/54 [==============================] - 0s 9ms/step - loss: 0.9938 - categorical_accuracy: 0.5518 - val_loss: 1.0839 - val_categorical_accuracy: 0.5067
Epoch 5/200
49/54 [==========================>...] - ETA: 0s - loss: 0.9409 - categorical_accuracy: 0.5957INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 138ms/step - loss: 0.9333 - categorical_accuracy: 0.5988 - val_loss: 0.9541 - val_categorical_accuracy: 0.6133
Epoch 6/200
50/54 [==========================>...] - ETA: 0s - loss: 0.8358 - categorical_accuracy: 0.6625INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 124ms/step - loss: 0.3345 - categorical_accuracy: 0.8918 - val_loss: 0.3223 - val_categorical_accuracy: 0.9200
Epoch 63/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3079 - categorical_accuracy: 0.9129 - val_loss: 0.3692 - val_categorical_accuracy: 0.9000
Epoch 64/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3408 - categorical_accuracy: 0.9059 - val_loss: 0.4948 - val_categorical_accuracy: 0.8533
Epoch 65/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3521 - categorical_accuracy: 0.8918 - val_loss: 0.3424 - val_categorical_accuracy: 0.9067
Epoch 66/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3299 - categorical_accuracy: 0.9035 - val_loss: 0.3868 - val_categorical_accuracy: 0.8800
Epoch 67/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3482 - categorical_accuracy: 0.8965 - val_loss: 0.3391 - val_categorical_accuracy: 0.8933
Epoch 68/200

INFO:tensorflow:Assets written to: models/AK-GRU256-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 137ms/step - loss: 0.2985 - categorical_accuracy: 0.9224 - val_loss: 0.2832 - val_categorical_accuracy: 0.9333
Epoch 119/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2929 - categorical_accuracy: 0.9129 - val_loss: 0.6488 - val_categorical_accuracy: 0.8067
Epoch 120/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3539 - categorical_accuracy: 0.8812 - val_loss: 0.3802 - val_categorical_accuracy: 0.9000
Epoch 121/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3106 - categorical_accuracy: 0.9071 - val_loss: 0.4405 - val_categorical_accuracy: 0.8667
Epoch 122/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2729 - categorical_accuracy: 0.9188 - val_loss: 0.3179 - val_categorical_accuracy: 0.9000
Epoch 123/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2867 - categorical_accuracy: 0.9176 - val_loss: 0.5926 - val_categorical_accuracy: 0.8067
Epoch 1

epoch/categorical_accuracy,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▅▆▆▅▆▆▆██▆██▇▇▆▇██▆▆▅██▇█▇▇█▇█▇██▇█▇██
epoch/val_loss,█▆▄▃▃▅▃▄▄▂▂▃▁▂▂▂▄▂▂▁▃▄▅▁▁▂▁▂▂▁▂▂▂▁▂▃▁▂▂▁
epoch/categorical_accuracy,0.95412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.19097
epoch/val_categorical_accuracy,0.92667


In [20]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.GRU(512, return_sequences=False, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001)))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-GRU512-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-GRU512-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 3s 8ms/step - loss: 1.2169 - categorical_accuracy: 0.4643

2024-04-22 12:35:11.908188: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15829989078708302192
2024-04-22 12:35:11.908244: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12174383135344089714
2024-04-22 12:35:11.908256: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14124334718774514054
2024-04-22 12:35:11.908279: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13537430977478141762
2024-04-22 12:35:12.531439: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5732312106366614420
2024-04-22 12:35:12.531575: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7579826450800086930


INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 10s 136ms/step - loss: 1.2074 - categorical_accuracy: 0.4671 - val_loss: 1.2623 - val_categorical_accuracy: 0.4200
Epoch 2/200
52/54 [===========================>..] - ETA: 0s - loss: 1.1401 - categorical_accuracy: 0.5228INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-cls=3.tf/assets


...

INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 125ms/step - loss: 0.3831 - categorical_accuracy: 0.8929 - val_loss: 0.3966 - val_categorical_accuracy: 0.9067
Epoch 44/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3708 - categorical_accuracy: 0.9059 - val_loss: 0.6723 - val_categorical_accuracy: 0.8200
Epoch 45/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3619 - categorical_accuracy: 0.9047 - val_loss: 0.4254 - val_categorical_accuracy: 0.9000
Epoch 46/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3896 - categorical_accuracy: 0.8859 - val_loss: 0.4094 - val_categorical_accuracy: 0.8800
Epoch 47/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3577 - categorical_accuracy: 0.9047 - val_loss: 0.5085 - val_categorical_accuracy: 0.8533
Epoch 48/200
54/54 [==============================] - 1s 10ms/step - loss: 0.4299 - categorical_accuracy: 0.8635 - val_loss: 0.4687 - val_categorical_accuracy: 0.8467
Epoch 49/200

INFO:tensorflow:Assets written to: models/AK-GRU512-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 7s 124ms/step - loss: 0.3202 - categorical_accuracy: 0.9141 - val_loss: 0.3758 - val_categorical_accuracy: 0.9400
Epoch 60/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3237 - categorical_accuracy: 0.9188 - val_loss: 0.4018 - val_categorical_accuracy: 0.9000
Epoch 61/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2980 - categorical_accuracy: 0.9282 - val_loss: 0.3755 - val_categorical_accuracy: 0.9000
Epoch 62/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3424 - categorical_accuracy: 0.9082 - val_loss: 0.4125 - val_categorical_accuracy: 0.9000
Epoch 63/200
54/54 [==============================] - 1s 9ms/step - loss: 0.3013 - categorical_accuracy: 0.9235 - val_loss: 0.4251 - val_categorical_accuracy: 0.8867
Epoch 64/200
54/54 [==============================] - 0s 9ms/step - loss: 0.3347 - categorical_accuracy: 0.9106 - val_loss: 0.4165 - val_categorical_accuracy: 0.9133
Epoch 65/200
5

epoch/categorical_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇██▇▇▇██▇▇▇▇▇▇█▇██▇█████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▆▆▅▅▆▅▇▇▇▇█▇▇█▇█▇█▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇
epoch/val_loss,█▅▄▃▄▅▂▅▂▁▂▂▁▂▂▁▂▁▁▁▃▁▂▂▃▂▃▂▂▂▃▁▂▃▂▂▂▂▁▃
epoch/categorical_accuracy,0.95294
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.1989
epoch/val_categorical_accuracy,0.84667


## Bidirectional LSTM

In [21]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM16-LR-Dense256-gelu-cls=3"),
            run_name="AK-BLSTM16-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 8ms/step - loss: 1.0840 - categorical_accuracy: 0.4105

2024-04-22 12:38:18.914113: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:38:18.914192: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 18252922663707797810
2024-04-22 12:38:18.914211: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:38:18.914267: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f2044da4f50>, 139805954698224), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff447c550>, 139775227457840), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1ff591fd90>, 139776573174080), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM16-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 415ms/step - loss: 0.1709 - categorical_accuracy: 0.9482 - val_loss: 0.2742 - val_categorical_accuracy: 0.9267
Epoch 195/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1977 - categorical_accuracy: 0.9435 - val_loss: 0.2974 - val_categorical_accuracy: 0.9200
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1757 - categorical_accuracy: 0.9518 - val_loss: 0.3225 - val_categorical_accuracy: 0.8867
Epoch 197/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2120 - categorical_accuracy: 0.9365 - val_loss: 0.3382 - val_categorical_accuracy: 0.8800
Epoch 198/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1648 - categorical_accuracy: 0.9553 - val_loss: 0.3465 - val_categorical_accuracy: 0.9000
Epoch 199/200
54/54 [==============================] - 1s 9ms/step - loss: 0.1867 - categorical_accuracy: 0.9553 - val_loss: 0.5068 - val_categorical_accuracy: 0.8733
Epoch

epoch/categorical_accuracy,▁▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█████▇█████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇███
epoch/val_loss,█▇▆▃▄▄▄▃▄▂▂▂▃▃▃▂▂▂▃▃▃▃▃▃▂▁▂▂▃▂▂▃▂▂▂▂▂▂▁▁
epoch/categorical_accuracy,0.94824
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18871
epoch/val_categorical_accuracy,0.91333


In [22]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM32-LR-Dense256-gelu-cls=3"),
            run_name="AK-BLSTM32-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.0742 - categorical_accuracy: 0.4216

2024-04-22 12:47:50.536797: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14635252564011947125
2024-04-22 12:47:50.536844: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 12:47:50.536871: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe733650>, 139775238372832), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 128), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbe7cba10>, 139774325937456), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbdb11d50>, 139774341087840), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM32-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 21s 393ms/step - loss: 0.2835 - categorical_accuracy: 0.9129 - val_loss: 0.3580 - val_categorical_accuracy: 0.9200
Epoch 76/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2899 - categorical_accuracy: 0.9153 - val_loss: 0.4352 - val_categorical_accuracy: 0.8533
Epoch 77/200
54/54 [==============================] - 1s 11ms/step - loss: 0.3016 - categorical_accuracy: 0.9224 - val_loss: 0.4281 - val_categorical_accuracy: 0.8667
Epoch 78/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2656 - categorical_accuracy: 0.9282 - val_loss: 0.4697 - val_categorical_accuracy: 0.8600
Epoch 79/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2553 - categorical_accuracy: 0.9294 - val_loss: 0.4806 - val_categorical_accuracy: 0.8800
Epoch 80/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2490 - categorical_accuracy: 0.9294 - val_loss: 0.4406 - val_categorical_accuracy: 0.8667
Epoch 81/

epoch/categorical_accuracy,▁▅▆▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▄▄▆▅▄▆▆▆▆▇▇▇▆▇▆█▇█▇▇█▆▇▇█▆▆▇▇▇▇██▇▇███
epoch/val_loss,█▅▆▅▄▅▆▂▄▃▄▂▃▂▃▃▂▁▂▂▃▂▂▄▂▃▃▄▄▂▃▂▂▂▁▃▂▁▁▁
epoch/categorical_accuracy,0.95412
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16424
epoch/val_categorical_accuracy,0.9


In [23]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM64-LR-Dense256-gelu-cls=3.tf"),
            run_name="AK-BLSTM64-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     51/Unknown - 5s 9ms/step - loss: 1.1174 - categorical_accuracy: 0.4645

2024-04-22 12:56:01.007349: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 12:56:01.007443: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 12:56:02.021207: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 12:56:02.021303: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9122064850495178467
2024-04-22 12:56:02.021348: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df5f310>, 139776572418720), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 256), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9d6a9550>, 139773748536496), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbc7f3ad0>, 139773393444400), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM64-LR-Dense256-gelu-cls=3.tf/assets


54/54 [==============================] - 23s 429ms/step - loss: 0.2175 - categorical_accuracy: 0.9471 - val_loss: 0.3553 - val_categorical_accuracy: 0.9267
Epoch 132/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2547 - categorical_accuracy: 0.9329 - val_loss: 0.4296 - val_categorical_accuracy: 0.8733
Epoch 133/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2217 - categorical_accuracy: 0.9494 - val_loss: 0.6109 - val_categorical_accuracy: 0.8667
Epoch 134/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2728 - categorical_accuracy: 0.9259 - val_loss: 0.4151 - val_categorical_accuracy: 0.8667
Epoch 135/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2595 - categorical_accuracy: 0.9282 - val_loss: 0.4296 - val_categorical_accuracy: 0.8933
Epoch 136/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2485 - categorical_accuracy: 0.9424 - val_loss: 0.3412 - val_categorical_accuracy: 0.9067
Epoch

epoch/categorical_accuracy,▁▄▅▆▆▆▇▆▇▇▇▇▇▇▇█████▇▇▇▇█▇▇▇█▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▅▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▄▆▆▄▄▅▅▅▇▇▇▇▇▇█▇▇▇▇▇█▆▆█▇▇▇▆▇██▇▇▇██▇██
epoch/val_loss,█▅▄▄█▇▄▆▅▂▃▂▂▃▂▁▂▂▂▃▂▃▄▂▁▂▂▁▃▂▁▁▂▁▁▁▂▁▂▂
epoch/categorical_accuracy,0.95765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18169
epoch/val_categorical_accuracy,0.89333


In [24]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM128-LR-Dense256-gelu-cls=3"),
            run_name="AK-BLSTM128-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     50/Unknown - 5s 9ms/step - loss: 1.1848 - categorical_accuracy: 0.4575

2024-04-22 13:03:09.148544: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:03:09.148639: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 13:03:09.148669: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7134357954231986336
2024-04-22 13:03:09.148747: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:03:10.147553: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1570638488140254319
2024-04-22 13:03:10.147610: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 13:03:10.147643: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv it

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9e53ab90>, 139773772335120), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f6018c8d0>, 139775234956720), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f9df37c50>, 139774320689760), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM128-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 23s 433ms/step - loss: 0.1622 - categorical_accuracy: 0.9753 - val_loss: 0.3317 - val_categorical_accuracy: 0.9267
Epoch 196/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1656 - categorical_accuracy: 0.9706 - val_loss: 0.3861 - val_categorical_accuracy: 0.9000
Epoch 197/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1566 - categorical_accuracy: 0.9741 - val_loss: 0.4553 - val_categorical_accuracy: 0.8733
Epoch 198/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1637 - categorical_accuracy: 0.9671 - val_loss: 0.4347 - val_categorical_accuracy: 0.9067
Epoch 199/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1584 - categorical_accuracy: 0.9682 - val_loss: 0.3544 - val_categorical_accuracy: 0.9000
Epoch 200/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1665 - categorical_accuracy: 0.9671 - val_loss: 0.4274 - val_categorical_accuracy: 0.8933


epoch/categorical_accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇█████▇▇▇▇▇▇▇▇▇██▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▅▇▆▇█▇▇▄▆▇██▇█▇▇▇▇▆▇▆▇▇▆▇▇▇▇█▇▇▇▇█████
epoch/val_loss,█▅▅▃▄▃▂▂▃▆▄▃▂▁▂▂▃▂▂▃▆▂▃▄▂▄▂▃▁▂▁▂▂▃▂▁▁▂▂▂
epoch/categorical_accuracy,0.96706
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.16646
epoch/val_categorical_accuracy,0.89333


In [25]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(LSTM(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BLSTM256-LR-Dense256-gelu-cls=3"),
            run_name="AK-BLSTM256-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 5s 9ms/step - loss: 1.2938 - categorical_accuracy: 0.4554

2024-04-22 13:10:18.669765: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c7593d0>, 139774286764992), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 1024), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3c78a490>, 139772389045552), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f859cc750>, 139772734880816), {}).


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BLSTM256-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 22s 406ms/step - loss: 0.2760 - categorical_accuracy: 0.9447 - val_loss: 0.4081 - val_categorical_accuracy: 0.9200
Epoch 88/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2635 - categorical_accuracy: 0.9565 - val_loss: 0.5574 - val_categorical_accuracy: 0.8600
Epoch 89/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2437 - categorical_accuracy: 0.9647 - val_loss: 0.4293 - val_categorical_accuracy: 0.9133
Epoch 90/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2807 - categorical_accuracy: 0.9459 - val_loss: 0.5564 - val_categorical_accuracy: 0.8800
Epoch 91/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2847 - categorical_accuracy: 0.9471 - val_loss: 0.4785 - val_categorical_accuracy: 0.8667
Epoch 92/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2562 - categorical_accuracy: 0.9600 - val_loss: 0.4919 - val_categorical_accuracy: 0.8867
Epoch 93/

epoch/categorical_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇██▇█████▇▇▇▇▇█▇▇████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▃▄▄▇▇▅▇▆▃▇▆▆███▇▇▇▇▄▇▄▆▆▇█▇▅▅▆▇▆▇▇▇█▇█
epoch/val_loss,█▅▇▆▅▃▄▅▂▄▇▃▃▃▂▂▃▃▃▂▁▇▂▆▃▃▃▁▂▆▅▃▂▃▂▃▂▂▁▂
epoch/categorical_accuracy,0.97529
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.15214
epoch/val_categorical_accuracy,0.89333


## Bidirectional GRU

In [26]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(16, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU16-LR-Dense256-gelu-cls=3"),
            run_name="AK-BGRU16-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     50/Unknown - 5s 8ms/step - loss: 1.0763 - categorical_accuracy: 0.4387

2024-04-22 13:16:48.065107: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:16:48.065172: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11254588046314640847
2024-04-22 13:16:48.965601: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 48), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f63c6f250>, 139774333344256), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 48), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f63c6f250>, 139774333344256), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 48), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1fbf8ca3d0>, 139773341549936), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1e98186910>, 139773763398080), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU16-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BGRU16-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 19s 363ms/step - loss: 0.1686 - categorical_accuracy: 0.9553 - val_loss: 0.2063 - val_categorical_accuracy: 0.9333
Epoch 188/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1808 - categorical_accuracy: 0.9471 - val_loss: 0.3015 - val_categorical_accuracy: 0.9133
Epoch 189/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1621 - categorical_accuracy: 0.9588 - val_loss: 0.3662 - val_categorical_accuracy: 0.9000
Epoch 190/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1703 - categorical_accuracy: 0.9447 - val_loss: 0.2615 - val_categorical_accuracy: 0.9200
Epoch 191/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1748 - categorical_accuracy: 0.9565 - val_loss: 0.3346 - val_categorical_accuracy: 0.9000
Epoch 192/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1658 - categorical_accuracy: 0.9494 - val_loss: 0.3104 - val_categorical_accuracy: 0.8933
Epoc

epoch/categorical_accuracy,▁▃▅▆▇▇▇▇▇▇▇▇▇▇▇██▇██▇▇▇▇▇▇█▇▇▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁
epoch/val_categorical_accuracy,▁▂▅▆▇▇▆▅▆▇▆▅▇▅▇▇█▇▇█▇▇▇▆▇▇▇▇▇██▆█▇██████
epoch/val_loss,█▆▄▄▃▃▃▄▃▂▂▄▂▃▂▂▁▂▂▁▂▃▂▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁
epoch/categorical_accuracy,0.95765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.15754
epoch/val_categorical_accuracy,0.91333


In [27]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(32, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU32-LR-Dense256-gelu-cls=3"),
            run_name="AK-BGRU32-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 7s 8ms/step - loss: 1.0738 - categorical_accuracy: 0.4503

2024-04-22 13:23:44.583272: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:23:44.583334: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:23:45.549775: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 13:23:45.549827: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17889825332423120889
2024-04-22 13:23:45.549844: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 13:23:45.549873: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4b8aab90>, 139773353871488), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4b8aab90>, 139773353871488), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32, 96), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f86c9d0d0>, 139773353870592), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f48c37550>, 139773372886320), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU32-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BGRU32-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 19s 367ms/step - loss: 0.1996 - categorical_accuracy: 0.9459 - val_loss: 0.2389 - val_categorical_accuracy: 0.9467
Epoch 155/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2039 - categorical_accuracy: 0.9329 - val_loss: 0.3764 - val_categorical_accuracy: 0.9000
Epoch 156/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1931 - categorical_accuracy: 0.9471 - val_loss: 0.4832 - val_categorical_accuracy: 0.8933
Epoch 157/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2328 - categorical_accuracy: 0.9341 - val_loss: 0.3835 - val_categorical_accuracy: 0.9067
Epoch 158/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1916 - categorical_accuracy: 0.9529 - val_loss: 0.4294 - val_categorical_accuracy: 0.8867
Epoch 159/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2027 - categorical_accuracy: 0.9576 - val_loss: 0.3089 - val_categorical_accuracy: 0.8933
Epoc

epoch/categorical_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇█▇███▇▇▇█▇█▇▇█▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▆▄▆▇▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇███▇▇
epoch/val_loss,█▇▄▆▃▃▄▄▃▃▂▂▂▂▂▂▃▂▂▂▄▂▂▂▂▃▂▂▂▂▁▃▂▂▃▂▂▂▃▂
epoch/categorical_accuracy,0.96
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.18387
epoch/val_categorical_accuracy,0.91333


In [28]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU64-LR-Dense256-gelu-cls=3"),
            run_name="AK-BGRU64-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     54/Unknown - 5s 8ms/step - loss: 1.0970 - categorical_accuracy: 0.4506

2024-04-22 13:31:20.816601: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1512661773567212545
2024-04-22 13:31:20.816647: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 13:31:20.816687: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 831169908037607112


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4a112910>, 139772388717744), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4a112910>, 139772388717744), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 192), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4a149c90>, 139772388710576), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f631b2dd0>, 139774333390240), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU64-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BGRU64-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 20s 369ms/step - loss: 0.2386 - categorical_accuracy: 0.9471 - val_loss: 0.3320 - val_categorical_accuracy: 0.9267
Epoch 97/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2199 - categorical_accuracy: 0.9494 - val_loss: 0.4277 - val_categorical_accuracy: 0.8733
Epoch 98/200
54/54 [==============================] - 1s 9ms/step - loss: 0.2616 - categorical_accuracy: 0.9400 - val_loss: 0.4857 - val_categorical_accuracy: 0.8733
Epoch 99/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2365 - categorical_accuracy: 0.9376 - val_loss: 0.4688 - val_categorical_accuracy: 0.8667
Epoch 100/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2420 - categorical_accuracy: 0.9400 - val_loss: 0.3634 - val_categorical_accuracy: 0.8600
Epoch 101/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2608 - categorical_accuracy: 0.9259 - val_loss: 0.3355 - val_categorical_accuracy: 0.9067
Epoch 102

epoch/categorical_accuracy,▁▄▆▆▆▆▇▇▇▇▇▇▇▇██▇███▇▇▇▇▇▇█▇█▇██████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▅▇▆▇▄▇▇▆▆▅█▇█▇████▇▇▆▇▅█▇▇▇▇█▇█▇███▇███
epoch/val_loss,█▅▄▃▃▇▃▂▄▅▇▂▂▃▃▂▂▃▂▃▂▄▂▅▂▃▃▃▂▂▂▂▂▃▂▂▁▁▁▁
epoch/categorical_accuracy,0.96353
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.14294
epoch/val_categorical_accuracy,0.90667


In [29]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(128, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU128-LR-Dense256-gelu-cls=3"),
            run_name="AK-BGRU128-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     54/Unknown - 7s 8ms/step - loss: 1.1600 - categorical_accuracy: 0.4812

2024-04-22 13:37:41.020576: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1297278236727296323
2024-04-22 13:37:41.020645: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:37:41.020663: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 13:37:41.020709: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:37:41.954243: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3924736971321085824


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4a20b5d0>, 139772352953936), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f4a20b5d0>, 139772352953936), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(128, 384), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f60c24c90>, 139772352949456), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f6039b210>, 139774302865648), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU128-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BGRU128-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 20s 379ms/step - loss: 0.1715 - categorical_accuracy: 0.9659 - val_loss: 0.3111 - val_categorical_accuracy: 0.9267
Epoch 188/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1654 - categorical_accuracy: 0.9671 - val_loss: 0.4324 - val_categorical_accuracy: 0.8733
Epoch 189/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1598 - categorical_accuracy: 0.9729 - val_loss: 0.4007 - val_categorical_accuracy: 0.8667
Epoch 190/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1740 - categorical_accuracy: 0.9635 - val_loss: 0.3180 - val_categorical_accuracy: 0.8800
Epoch 191/200
54/54 [==============================] - 1s 10ms/step - loss: 0.1537 - categorical_accuracy: 0.9706 - val_loss: 0.3547 - val_categorical_accuracy: 0.8933
Epoch 192/200
54/54 [==============================] - 1s 11ms/step - loss: 0.1567 - categorical_accuracy: 0.9671 - val_loss: 0.4460 - val_categorical_accuracy: 0.8933
Epoc

epoch/categorical_accuracy,▁▄▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇█▇▇▇▇▇▇▇█▇███████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▁▃▆▅▆▅▅▅▅▇▆▇▅▇█▇█▇█▇▅▆▅▇▆▇▇█▆▆▆▇▇▇▇██▇█▇
epoch/val_loss,█▅▄▄▃▄▃▅▅▂▂▃▃▃▂▂▂▂▁▂▆▅▅▂▃▂▁▁▃▂▃▂▁▁▂▂▁▂▂▃
epoch/categorical_accuracy,0.97765
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.1541
epoch/val_categorical_accuracy,0.88


In [30]:
from tensorflow.keras import layers

model = Sequential()
#x,y,z -> y,z as the input shape
model.add(layers.Bidirectional(layers.GRU(256, return_sequences=True, input_shape=(CONFIG.VIDEO_LENGTH, 1629),
               kernel_regularizer=l2(0.0001), 
               activity_regularizer=l2(0.0001))))
model.add(layers.Flatten())
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(256, activation='gelu'))
model.add(Dense(len(train_dataset_parquet.unique_labels), activation='softmax'))


keras_train(model, filepath=os.path.join("models", "AK-BGRU256-LR-Dense256-gelu-cls=3"),
            run_name="AK-BGRU256-LR-Dense256-gelu-cls=3",
            USE_WANDB=True)

Epoch 1/200
     49/Unknown - 5s 9ms/step - loss: 1.2978 - categorical_accuracy: 0.4668

2024-04-22 13:44:35.675823: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2737983808172683809
2024-04-22 13:44:35.675891: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8866511902849424037
2024-04-22 13:44:35.675907: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13721100163777961033
2024-04-22 13:44:35.675945: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6419013273699342798
2024-04-22 13:44:36.595766: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15496870552709104721
2024-04-22 13:44:36.595827: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1033902641938270650
2024-04-22 13:44:36.595840: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3d43dc90>, 139772749676928), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1629, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3d43dc90>, 139772749676928), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 768), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f3d4ce990>, 139773383338928), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1f49a1b290>, 139772374128224), {}).


INFO:tensorflow:Assets written to: models/AK-BGRU256-LR-Dense256-gelu-cls=3/assets


INFO:tensorflow:Assets written to: models/AK-BGRU256-LR-Dense256-gelu-cls=3/assets


54/54 [==============================] - 20s 379ms/step - loss: 0.2938 - categorical_accuracy: 0.9400 - val_loss: 0.3752 - val_categorical_accuracy: 0.9200
Epoch 62/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3223 - categorical_accuracy: 0.9329 - val_loss: 0.6144 - val_categorical_accuracy: 0.8333
Epoch 63/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2947 - categorical_accuracy: 0.9400 - val_loss: 0.4718 - val_categorical_accuracy: 0.8600
Epoch 64/200
54/54 [==============================] - 1s 10ms/step - loss: 0.3049 - categorical_accuracy: 0.9388 - val_loss: 0.4611 - val_categorical_accuracy: 0.8733
Epoch 65/200
54/54 [==============================] - 1s 11ms/step - loss: 0.2752 - categorical_accuracy: 0.9400 - val_loss: 0.4752 - val_categorical_accuracy: 0.8667
Epoch 66/200
54/54 [==============================] - 1s 10ms/step - loss: 0.2889 - categorical_accuracy: 0.9459 - val_loss: 0.4502 - val_categorical_accuracy: 0.8667
Epoch 67/

epoch/categorical_accuracy,▁▄▆▆▆▆▇▇▇▇▇▇▇▇▇▇████▇▇▇▇▇▇▇█████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
epoch/loss,█▆▄▄▄▄▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁
epoch/val_categorical_accuracy,▂▂▄▁▄▃▄▅▄▄▆▅▅▆▆█▇▆▆▇▇▆▅▇▇▆▅▇▅▆▆▅▆▇▇▇▇▆█▇
epoch/val_loss,██▄▇▅▆▆▄▃▄▃▃▅▃▃▁▃▃▃▂▃▃▅▃▂▃▄▂▄▂▃▃▂▃▂▂▂▂▁▂
epoch/categorical_accuracy,0.98353
epoch/epoch,199
epoch/learning_rate,3e-05
epoch/loss,0.13435
epoch/val_categorical_accuracy,0.9
